In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import yaml
from tqdm import tqdm
from pathlib import Path
from glob import glob
from functools import partial
from multiprocessing import Pool

import shapefile
import shapely
from shapely.geometry import Point

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from projections.shapefiles import load_shapes, iter_records
from projections.models import Records
from projections import raster, utils

pd.set_option('max_columns', None)

/home/luisdasilva/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
output_folder = Path('../Output/Precipitaciones/Raster/CRU/')

In [3]:
df = pd.read_csv('../Data/Precipitaciones/cru_80-20.csv')

print(df.shape)

lon = 'lon'
lat = 'lat'
country = 'country'
locs = df  # [[lon, lat]].drop_duplicates()

df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']))
df = raster.create_by_separation(df, lat=lat, lon=lon)

df.head()

(67420, 494)
Separation: {'lat': 0.5, 'lon': 0.5}


,lon,lat,80-01,80-02,80-03,80-04,80-05,80-06,80-07,80-08,80-09,80-10,80-11,80-12,81-01,81-02,81-03,81-04,81-05,81-06,81-07,81-08,81-09,81-10,81-11,81-12,82-01,82-02,82-03,82-04,82-05,82-06,82-07,82-08,82-09,82-10,82-11,82-12,83-01,83-02,83-03,83-04,83-05,83-06,83-07,83-08,83-09,83-10,83-11,83-12,84-01,84-02,84-03,84-04,84-05,84-06,84-07,84-08,84-09,84-10,84-11,84-12,85-01,85-02,85-03,85-04,85-05,85-06,85-07,85-08,85-09,85-10,85-11,85-12,86-01,86-02,86-03,86-04,86-05,86-06,86-07,86-08,86-09,86-10,86-11,86-12,87-01,87-02,87-03,87-04,87-05,87-06,87-07,87-08,87-09,87-10,87-11,87-12,88-01,88-02,88-03,88-04,88-05,88-06,88-07,88-08,88-09,88-10,88-11,88-12,89-01,89-02,89-03,89-04,89-05,89-06,89-07,89-08,89-09,89-10,89-11,89-12,90-01,90-02,90-03,90-04,90-05,90-06,90-07,90-08,90-09,90-10,90-11,90-12,91-01,91-02,91-03,91-04,91-05,91-06,91-07,91-08,91-09,91-10,91-11,91-12,92-01,92-02,92-03,92-04,92-05,92-06,92-07,92-08,92-09,92-10,92-11,92-12,93-01,93-02,93-03,93-04,93-05,93-06,93-07,93-08,93-09,93-10,93-11,93-12,94-01,94-02,94-03,94-04,94-05,94-06,94-07,94-08,94-09,94-10,94-11,94-12,95-01,95-02,95-03,95-04,95-05,95-06,95-07,95-08,95-09,95-10,95-11,95-12,96-01,96-02,96-03,96-04,96-05,96-06,96-07,96-08,96-09,96-10,96-11,96-12,97-01,97-02,97-03,97-04,97-05,97-06,97-07,97-08,97-09,97-10,97-11,97-12,98-01,98-02,98-03,98-04,98-05,98-06,98-07,98-08,98-09,98-10,98-11,98-12,99-01,99-02,99-03,99-04,99-05,99-06,99-07,99-08,99-09,99-10,99-11,99-12,00-01,00-02,00-03,00-04,00-05,00-06,00-07,00-08,00-09,00-10,00-11,00-12,01-01,01-02,01-03,01-04,01-05,01-06,01-07,01-08,01-09,01-10,01-11,01-12,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,02-11,02-12,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,03-09,03-10,03-11,03-12,04-01,04-02,04-03,04-04,04-05,04-06,04-07,04-08,04-09,04-10,04-11,04-12,05-01,05-02,05-03,05-04,05-05,05-06,05-07,05-08,05-09,05-10,05-11,05-12,06-01,06-02,06-03,06-04,06-05,06-06,06-07,06-08,06-09,06-10,06-11,06-12,07-01,07-02,07-03,07-04,07-05,07-06,07-07,07-08,07-09,07-10,07-11,07-12,08-01,08-02,08-03,08-04,08-05,08-06,08-07,08-08,08-09,08-10,08-11,08-12,09-01,09-02,09-03,09-04,09-05,09-06,09-07,09-08,09-09,09-10,09-11,09-12,10-01,10-02,10-03,10-04,10-05,10-06,10-07,10-08,10-09,10-10,10-11,10-12,11-01,11-02,11-03,11-04,11-05,11-06,11-07,11-08,11-09,11-10,11-11,11-12,12-01,12-02,12-03,12-04,12-05,12-06,12-07,12-08,12-09,12-10,12-11,12-12,13-01,13-02,13-03,13-04,13-05,13-06,13-07,13-08,13-09,13-10,13-11,13-12,14-01,14-02,14-03,14-04,14-05,14-06,14-07,14-08,14-09,14-10,14-11,14-12,15-01,15-02,15-03,15-04,15-05,15-06,15-07,15-08,15-09,15-10,15-11,15-12,16-01,16-02,16-03,16-04,16-05,16-06,16-07,16-08,16-09,16-10,16-11,16-12,17-01,17-02,17-03,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12,18-01,18-02,18-03,18-04,18-05,18-06,18-07,18-08,18-09,18-10,18-11,18-12,19-01,19-02,19-03,19-04,19-05,19-06,19-07,19-08,19-09,19-10,19-11,19-12,20-01,20-02,20-03,20-04,20-05,20-06,20-07,20-08,20-09,20-10,20-11,20-12,geometry,raster
0,-68.25,-55.75,59.000000,141.900009,135.100006,51.200001,59.200001,75.000000,107.200005,69.500000,103.000000,73.700005,66.000000,60.000000,61.400002,67.900002,241.699997,98.400002,125.500000,97.300003,136.000000,141.100006,87.000000,72.300003,41.500000,76.700005,92.700005,99.000000,57.100002,70.900002,64.900002,58.799999,84.200005,46.400002,72.099998,134.800003,83.000000,43.200001,81.400002,90.200005,80.900002,78.500000,67.000000,78.900002,32.000000,64.000000,36.400002,59.799999,45.200001,28.200001,88.900002,52.700001,59.400002,104.300003,160.900009,59.400002,67.599998,80.400002,39.799999,85.000000,45.299999,40.100002,55.200001,73.700005,96.700005,50.000000,50.000000,61.799999,34.200001,37.799999,82.000000,45.600002,78.200005,73.800003,41.100002,93.000000,67.500000,72.800003,122.099998,103.300003,104.599998,29.000000,45.500000,42.100002,116.599998,90.500000,92.099998,111.800003,64.300003,104.200005,42.600002,89.300003,112.700005,31.400000,60.200001,69.599998,73.099998,54.400002,94.599998,77.800003,94.200005

# Map raster to adm2

In [4]:
shapes = load_shapes('../Shapefiles/preprocessed/')

Loading shapes: 100%|██████████| 276/276 [02:22<00:00,  1.94it/s]


In [15]:
# Find intersections per shape
locs = []
for pol, record in iter_records(shapes):
    subdf = raster.intersection_ratio(df, pol)
    
    if subdf is None or subdf.empty:
        continue
        
    for i in range(len(record)):
        subdf[f'adm{i}'] = record[i]
        
    locs.append(subdf)
    
locs = locs[0].append(locs[1:], ignore_index=True)
locs.drop_duplicates(inplace=True)
locs.to_csv(output_folder / 'loc_map.csv', index=False)
print(locs.shape)
locs.head()

(284132, 6)


,lat,lon,intersection_area,adm0,adm1,adm2
0,12.25,-70.25,2.209819e+06,ABW,,
1,36.25,70.75,0.000000e+00,AFG,AFG.1_1,AFG.1.1_1
2,36.25,71.25,1.537801e+07,AFG,AFG.1_1,AFG.1.1_1
3,36.75,70.75,9.136605e+07,AFG,AFG.1_1,AFG.1.1_1
4,36.75,71.25,1.127589e+09,AFG,AFG.1_1,AFG.1.1_1


# Map locations back

In [16]:
locs = pd.read_csv(output_folder / 'loc_map.csv')
df = df.merge(locs, on=['lat', 'lon'])

In [17]:
pivot = raster.weighted_pivot(
    df, 
    weight='intersection_area', 
    value_name='precipitation', 
    id_vars=['adm0', 'adm1', 'adm2']
)
utils.map_year_month(pivot, 'time', 1980, 2020)

# Save results
pivot.to_csv(output_folder / 'CRU.csv', index=False)
print(pivot.shape)
pivot.head()

(22547376, 9)


,adm0,adm1,adm2,time,precipitation,intersection_area_count,intersection_area_sum,year,month
0,AFG,AFG.10_1,AFG.10.1_1,80-01,399.087498,15,1.107648e+10,1980,1
45828,AFG,AFG.10_1,AFG.10.1_1,80-02,495.563053,15,1.107648e+10,1980,2
91656,AFG,AFG.10_1,AFG.10.1_1,80-03,672.722962,15,1.107648e+10,1980,3
137484,AFG,AFG.10_1,AFG.10.1_1,80-04,145.259993,15,1.107648e+10,1980,4
183312,AFG,AFG.10_1,AFG.10.1_1,80-05,15.781008,15,1.107648e+10,1980,5


# Aggregate

In [18]:
df = pd.read_csv(output_folder / 'CRU.csv')

time_groups = {'yearly': ['year'], 'monthly': ['year', 'month']}
loc_groups = {'country': ['adm0'], 
              'edo': ['adm0', 'adm1'], 
              'mun': ['adm0', 'adm1', 'adm2']}

groups = {}
for loc_name, loc_group in loc_groups.items():
    for time_name, time_group in time_groups.items():
        groups[f'{loc_name}_{time_name}'] = loc_group + time_group

output_folder = Path('../Output/Precipitaciones/Raster/CRU/')
done = [Path(x).name for x in glob(str(output_folder / 'CRU_*.csv'))]
    
utils.aggregate_by_groups(df, groups, output_folder, ['precipitation'], done=done, batch=None)


Aggregating: 100%|██████████| 6/6 [10:15<00:00, 102.57s/it]


# Ethnic

In [4]:
# Read shapes
adm = gpd.read_file('../Shapefiles/ethnic_preprocessed/tribe_adm0.shp')
adm.head()

,NAME,TRIBE_CODE,LAT,LON,GID_0,NAME_0,area_tribe,area_adm,area_inter,geometry
0,GUANCHE,250,28.335400,-15.673500,ESP,Spain,7.485274e+09,5.060438e+11,7.112255e+09,"MULTIPOLYGON (((-17.89487 27.78681, -17.89514 ..."
1,JEBALA,312,34.850600,-5.280360,ESP,Spain,1.673756e+10,5.060438e+11,2.028542e+07,"MULTIPOLYGON (((-5.37708 35.91704, -5.37708 35..."
2,RIF,651,34.790800,-3.718290,ESP,Spain,2.027292e+10,5.060438e+11,7.560911e+06,"MULTIPOLYGON (((-2.92593 35.29208, -2.92708 35..."
3,ADANGME,6,6.076851,0.270457,GHA,Ghana,4.986379e+09,2.383243e+11,4.803454e+09,"MULTIPOLYGON (((0.69465 5.77336, 0.69328 5.775..."
4,ADELE,8,8.244284,0.673651,GHA,Ghana,1.413803e+09,2.383243e+11,6.779161e+08,"POLYGON ((0.45975 8.06680, 0.46512 8.07837, 0...."


In [5]:
# Find intersections per shape
ethnic_df = []
for idx, row in tqdm(adm.iterrows(), total=adm.shape[0]):
    subdf = raster.intersection_ratio(df, row['geometry'])
    
    if subdf is None or subdf.empty:
        continue
        
    for col in ('NAME', 'TRIBE_CODE', 'GID_0'):
        subdf[col] = row[col]
        
    ethnic_df.append(subdf)
    
ethnic_df = ethnic_df[0].append(ethnic_df[1:], ignore_index=True)
ethnic_df.to_csv(output_folder / 'ethnic_map.csv', index=False)

100%|██████████| 1412/1412 [08:54<00:00,  2.64it/s]


In [6]:
# Merge locations back
# ethnic_df = pd.read_csv(output_folder / 'ethnic_map.csv')
df = df.merge(ethnic_df, on=['lat', 'lon'])
df.head()

,lon,lat,80-01,80-02,80-03,80-04,80-05,80-06,80-07,80-08,80-09,80-10,80-11,80-12,81-01,81-02,81-03,81-04,81-05,81-06,81-07,81-08,81-09,81-10,81-11,81-12,82-01,82-02,82-03,82-04,82-05,82-06,82-07,82-08,82-09,82-10,82-11,82-12,83-01,83-02,83-03,83-04,83-05,83-06,83-07,83-08,83-09,83-10,83-11,83-12,84-01,84-02,84-03,84-04,84-05,84-06,84-07,84-08,84-09,84-10,84-11,84-12,85-01,85-02,85-03,85-04,85-05,85-06,85-07,85-08,85-09,85-10,85-11,85-12,86-01,86-02,86-03,86-04,86-05,86-06,86-07,86-08,86-09,86-10,86-11,86-12,87-01,87-02,87-03,87-04,87-05,87-06,87-07,87-08,87-09,87-10,87-11,87-12,88-01,88-02,88-03,88-04,88-05,88-06,88-07,88-08,88-09,88-10,88-11,88-12,89-01,89-02,89-03,89-04,89-05,89-06,89-07,89-08,89-09,89-10,89-11,89-12,90-01,90-02,90-03,90-04,90-05,90-06,90-07,90-08,90-09,90-10,90-11,90-12,91-01,91-02,91-03,91-04,91-05,91-06,91-07,91-08,91-09,91-10,91-11,91-12,92-01,92-02,92-03,92-04,92-05,92-06,92-07,92-08,92-09,92-10,92-11,92-12,93-01,93-02,93-03,93-04,93-05,93-06,93-07,93-08,93-09,93-10,93-11,93-12,94-01,94-02,94-03,94-04,94-05,94-06,94-07,94-08,94-09,94-10,94-11,94-12,95-01,95-02,95-03,95-04,95-05,95-06,95-07,95-08,95-09,95-10,95-11,95-12,96-01,96-02,96-03,96-04,96-05,96-06,96-07,96-08,96-09,96-10,96-11,96-12,97-01,97-02,97-03,97-04,97-05,97-06,97-07,97-08,97-09,97-10,97-11,97-12,98-01,98-02,98-03,98-04,98-05,98-06,98-07,98-08,98-09,98-10,98-11,98-12,99-01,99-02,99-03,99-04,99-05,99-06,99-07,99-08,99-09,99-10,99-11,99-12,00-01,00-02,00-03,00-04,00-05,00-06,00-07,00-08,00-09,00-10,00-11,00-12,01-01,01-02,01-03,01-04,01-05,01-06,01-07,01-08,01-09,01-10,01-11,01-12,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,02-11,02-12,03-01,03-02,03-03,03-04,03-05,03-06,03-07,03-08,03-09,03-10,03-11,03-12,04-01,04-02,04-03,04-04,04-05,04-06,04-07,04-08,04-09,04-10,04-11,04-12,05-01,05-02,05-03,05-04,05-05,05-06,05-07,05-08,05-09,05-10,05-11,05-12,06-01,06-02,06-03,06-04,06-05,06-06,06-07,06-08,06-09,06-10,06-11,06-12,07-01,07-02,07-03,07-04,07-05,07-06,07-07,07-08,07-09,07-10,07-11,07-12,08-01,08-02,08-03,08-04,08-05,08-06,08-07,08-08,08-09,08-10,08-11,08-12,09-01,09-02,09-03,09-04,09-05,09-06,09-07,09-08,09-09,09-10,09-11,09-12,10-01,10-02,10-03,10-04,10-05,10-06,10-07,10-08,10-09,10-10,10-11,10-12,11-01,11-02,11-03,11-04,11-05,11-06,11-07,11-08,11-09,11-10,11-11,11-12,12-01,12-02,12-03,12-04,12-05,12-06,12-07,12-08,12-09,12-10,12-11,12-12,13-01,13-02,13-03,13-04,13-05,13-06,13-07,13-08,13-09,13-10,13-11,13-12,14-01,14-02,14-03,14-04,14-05,14-06,14-07,14-08,14-09,14-10,14-11,14-12,15-01,15-02,15-03,15-04,15-05,15-06,15-07,15-08,15-09,15-10,15-11,15-12,16-01,16-02,16-03,16-04,16-05,16-06,16-07,16-08,16-09,16-10,16-11,16-12,17-01,17-02,17-03,17-04,17-05,17-06,17-07,17-08,17-09,17-10,17-11,17-12,18-01,18-02,18-03,18-04,18-05,18-06,18-07,18-08,18-09,18-10,18-11,18-12,19-01,19-02,19-03,19-04,19-05,19-06,19-07,19-08,19-09,19-10,19-11,19-12,20-01,20-02,20-03,20-04,20-05,20-06,20-07,20-08,20-09,20-10,20-11,20-12,geometry,raster,intersection_area,NAME,TRIBE_CODE,GID_0
0,19.25,-34.75,22.300001,23.0,3.700000,57.299999,102.400002,99.800003,20.100000,77.300003,46.299999,37.000000,125.099998,63.500000,118.099998,18.300001,59.799999,84.400002,23.0,74.099998,131.400009,95.599998,71.000000,25.500000,35.700001,26.000000,41.900002,11.900001,20.800001,110.700005,27.500000,73.000000,54.100002,93.200005,22.400000,34.900002,25.300001,31.600000,8.400001,56.100002,45.400002,20.200001,121.200005,133.699997,80.900002,49.299999,38.600002,18.200001,26.000000,11.000000,14.500000,9.900001,44.200001,42.200001,146.000000,39.299999,62.500000,31.800001,110.000000,80.200005,15.400001,79.500000,46.600002,38.600002,73.099998,66.599998,33.400002,84.099998,146.600006,74.800003,33.799999,41.299999,21.200001,35.200001,11.600000,31.400000,46.500000,48.000000,25.700001,98.599998,104.400002,175.900009,48.299999,41.000000,54.900002,19.100000,41.000000,24.200001,21.700001,63.200001,110.099998,78.099998,114.599998,120.800003,65.500000,13.7,17.9,46.900002,2.7,8.900001,21.

In [7]:
# Pivot to long
pivot = raster.weighted_pivot(
    df, 
    weight='intersection_area', 
    value_name='precipitation', 
    id_vars=('NAME', 'TRIBE_CODE', 'GID_0')
)
utils.map_year_month(pivot, 'time', 1980, 2020)


# Save results
pivot.to_csv(output_folder / 'CRU_Ethnic.csv', index=False)
# print(melt.shape)
# pivot.head()

NameError: name 'melt' is not defined

In [4]:
pivot = pd.read_csv(output_folder / 'CRU_Ethnic.csv')

# Aggregate
time_groups = {'yearly': ['year'], 'monthly': ['year', 'month']}
loc_groups = {'ethnic': ['NAME', 'TRIBE_CODE', 'GID_0']}

groups = {}
for loc_name, loc_group in loc_groups.items():
    for time_name, time_group in time_groups.items():
        groups[f'{loc_name}_{time_name}'] = loc_group + time_group

output_folder = Path('../Output/Precipitaciones/Raster/CRU/')

done = [Path(x).name for x in glob(str(output_folder / 'CRU_*.csv'))]
    
utils.aggregate_by_groups(pivot, groups, output_folder, ['precipitation'], done=done, batch=None)

Aggregating: 100%|██████████| 2/2 [00:07<00:00,  3.80s/it]
